# SA-Co/VEval 評価 (Evaluation)

このノートブックでは、SA-Co/VEvalデータセットに対する評価を実行する方法を示します。
評価には `VEvalEvaluator` クラスを使用し、予測結果 (Predictions) と正解データ (Ground Truth) を比較します。

## Google Colab セットアップ

Google Colabで実行する場合は、以下のセルを実行してください。
これにより、必要なライブラリのインストールと、データの読み込み準備が行われます。

In [ ]:
# Google Colab環境のセットアップ
try:
    import google.colab
    IN_COLAB = True
    print("Google Colab環境で実行中")
except ImportError:
    IN_COLAB = False
    print("ローカル環境で実行中")

if IN_COLAB:
    # 1. Google Driveをマウント (推奨)
    # 自分のデータやコードを使用する場合は、Google Driveにアップロードしてマウントします
    from google.colab import drive
    drive.mount('/content/drive')
    
    # 2. SAM3のインストールとパス設定
    import os
    import sys
    
    # Google Drive内のSAM3ディレクトリのパス
    # ※ご自身の環境に合わせてパスを変更してください
    DRIVE_SAM3_PATH = "/content/drive/MyDrive/sam3"
    
    if os.path.exists(DRIVE_SAM3_PATH):
        print(f"Google Drive内のSAM3が見つかりました: {DRIVE_SAM3_PATH}")
        os.chdir(DRIVE_SAM3_PATH)
        print("カレントディレクトリを変更しました。")
        
        # 依存関係のインストール
        print("依存関係をインストールしています...")
        # Numpy 2.0との互換性問題を回避するためにバージョンを固定
        !pip install -q "numpy<2.0"
        !pip install -q -e .
        !pip install -q pycocotools
        
    else:
        print(f"Google Drive内に {DRIVE_SAM3_PATH} が見つかりませんでした。")
        print("GitHubからSAM3をクローンしてインストールします...")
        
        # GitHubからクローン
        if not os.path.exists("/content/sam3"):
            !git clone https://github.com/facebookresearch/sam3.git /content/sam3
            
        os.chdir("/content/sam3")
        print("カレントディレクトリを /content/sam3 に変更しました。")
        
        # 依存関係のインストール
        # Numpy 2.0との互換性問題を回避するためにバージョンを固定
        !pip install -q "numpy<2.0"
        !pip install -q -e .
        !pip install -q pycocotools
    
    print("セットアップが完了しました。")
    print(f"現在の作業ディレクトリ: {os.getcwd()}")

### データの使用について

- **Google Driveを使用する場合**: `GT_DIR` や `PRED_DIR` には `/content/drive/MyDrive/...` から始まるパスを指定してください。
- **GitHubからクローンした場合**: 左側のファイルブラウザから `/content/sam3` 内を確認できます。データは別途アップロードが必要です。

In [ ]:
# Google Colab環境のセットアップ
try:
    import google.colab
    IN_COLAB = True
    print("Google Colab環境で実行中")
except ImportError:
    IN_COLAB = False
    print("ローカル環境で実行中")

if IN_COLAB:
    # 1. Google Driveをマウント (推奨)
    # 自分のデータやコードを使用する場合は、Google Driveにアップロードしてマウントします
    from google.colab import drive
    drive.mount('/content/drive')
    
    # 2. SAM3のインストールとパス設定
    import os
    import sys
    
    # Google Drive内のSAM3ディレクトリのパス
    # ※ご自身の環境に合わせてパスを変更してください
    DRIVE_SAM3_PATH = "/content/drive/MyDrive/sam3"
    
    if os.path.exists(DRIVE_SAM3_PATH):
        print(f"Google Drive内のSAM3が見つかりました: {DRIVE_SAM3_PATH}")
        os.chdir(DRIVE_SAM3_PATH)
        print("カレントディレクトリを変更しました。")
        
        # 依存関係のインストール
        print("依存関係をインストールしています...")
        !pip install -q -e .
        !pip install -q pycocotools
        
    else:
        print(f"Google Drive内に {DRIVE_SAM3_PATH} が見つかりませんでした。")
        print("GitHubからSAM3をクローンしてインストールします...")
        
        # GitHubからクローン
        if not os.path.exists("/content/sam3"):
            !git clone https://github.com/facebookresearch/sam3.git /content/sam3
            
        os.chdir("/content/sam3")
        print("カレントディレクトリを /content/sam3 に変更しました。")
        
        # 依存関係のインストール
        !pip install -q -e .
        !pip install -q pycocotools
    
    print("セットアップが完了しました。")
    print(f"現在の作業ディレクトリ: {os.getcwd()}")

### データの使用について

- **Google Driveを使用する場合**: `GT_DIR` や `PRED_DIR` には `/content/drive/MyDrive/...` から始まるパスを指定してください。
- **GitHubからクローンした場合**: 左側のファイルブラウザから `/content/sam3` 内を確認できます。データは別途アップロードが必要です。

## ライブラリのインポート

必要なライブラリと評価用クラスをインポートします。

In [ ]:
import json
import os

from sam3.eval.saco_veval_eval import VEvalEvaluator

## 設定

評価対象のデータセットと、ファイルパスを設定します。
ユーザーは `GT_DIR` (正解データのパス) と `PRED_DIR` (予測結果のパス) を自身の環境に合わせて変更する必要があります。

In [ ]:
DATASETS_TO_EVAL = [
    "saco_veval_sav_test",
    "saco_veval_yt1b_test",
    "saco_veval_smartglasses_test",
]
# 正解アノテーション(GT)と予測結果(PRED)ファイルが存在するディレクトリに更新してください
GT_DIR = None # ここにアノテーションのパスを入力
PRED_DIR = None # ここに画像のパスを入力

## 評価の実行

各データセットに対して評価を実行します。
すでに評価結果ファイルが存在する場合はそれを読み込み、存在しない場合は `VEvalEvaluator` を使用して評価を実行し、結果を保存します。

In [ ]:
all_eval_res = {}
for dataset_name in DATASETS_TO_EVAL:
    gt_annot_file = os.path.join(GT_DIR, dataset_name + ".json")
    pred_file = os.path.join(PRED_DIR, dataset_name + "_preds.json")
    eval_res_file = os.path.join(PRED_DIR, dataset_name + "_eval_res.json")

    if os.path.exists(eval_res_file):
        with open(eval_res_file, "r") as f:
            eval_res = json.load(f)
    else:
        # あるいは、sam3/eval/saco_veval_eval.py を活用して
        # 評価器(evaluator)をオフラインで先に実行することも可能です
        print(f"=== Running evaluation for Pred {pred_file} vs GT {gt_annot_file} ===")
        veval_evaluator = VEvalEvaluator(
            gt_annot_file=gt_annot_file, eval_res_file=eval_res_file
        )
        eval_res = veval_evaluator.run_eval(pred_file=pred_file)
        print(f"=== Results saved to {eval_res_file} ===")

    all_eval_res[dataset_name] = eval_res

## 結果の表示

評価結果から主要な指標 (Metrics) を抽出し、テーブル形式で表示します。

- **cgf1**: Class-agnostic Generalized F1 score (クラス非依存の一般化F1スコア)
- **pHOTA**: phrase HOTA (Higher Order Tracking Accuracy) - フレーズごとの追跡精度

In [ ]:
REPORT_METRICS = {
    "video_mask_demo_cgf1_micro_50_95": "cgf1",
    "video_mask_all_phrase_HOTA": "pHOTA",
}

In [ ]:
res_to_print = []
for dataset_name in DATASETS_TO_EVAL:
    eval_res = all_eval_res[dataset_name]
    row = [dataset_name]
    for metric_k, metric_v in REPORT_METRICS.items():
        row.append(eval_res["dataset_results"][metric_k])
    res_to_print.append(row)

# データセットヘッダーを表示 (各データセットは2つの指標にまたがる: 13 + 3 + 13 = 29文字)
print("| " + " | ".join(f"{ds:^29}" for ds in DATASETS_TO_EVAL) + " |")

# 指標ヘッダーを表示
metrics = list(REPORT_METRICS.values())
print("| " + " | ".join(f"{m:^13}" for _ in DATASETS_TO_EVAL for m in metrics) + " |")

# 評価結果を表示
values = []
for row in res_to_print:
    values.extend([f"{v * 100:^13.1f}" for v in row[1:]])
print("| " + " | ".join(values) + " |")